

 # 训练状态的断点保存与加载 (Checkpointing and loading training states)



 本示例演示NeuralOP库中``Trainer``类的保存和加载功能，

 该功能可以轻松实现训练状态的断点保存（checkpoint）和恢复训练（resume training）。



 ## 导入依赖库



In [9]:
# 导入PyTorch核心库，用于张量计算和深度学习
import torch
# 导入matplotlib绘图库（本示例未实际使用，为预留可视化接口）
import matplotlib.pyplot as plt
# 导入系统库，用于刷新输出流
import sys
# 从neuralop.models导入FNO（Fourier Neural Operator）模型，这是神经算子的核心模型
from neuralop.models import FNO
# 从neuralop导入Trainer训练器，用于封装训练流程
from neuralop import Trainer
# 从neuralop.training导入AdamW优化器（带权重衰减的Adam）
from neuralop.training import AdamW
# 从neuralop数据模块加载Darcy Flow（达西流）小型数据集，这是流体力学的经典基准数据集
from neuralop.data.datasets import load_darcy_flow_small
# 导入参数计数工具，用于统计模型参数量
from neuralop.utils import count_model_params
# 导入损失函数：LpLoss（Lp范数损失）和H1Loss（H1范数损失，包含函数值和梯度的损失）
from neuralop import LpLoss, H1Loss

# 设置训练设备，cpu表示使用CPU训练（若有GPU可改为"cuda"）
device = "cpu"




 ## 加载Darcy-Flow（达西流）数据集

 达西流数据集是用于测试神经算子性能的经典流体力学数据集，

 输入为渗透率场，输出为压力场



In [10]:
# 加载小型达西流数据集，并返回训练加载器、测试加载器和数据处理器
train_loader, test_loaders, data_processor = load_darcy_flow_small(
    n_train=1000,          # 训练集样本数量：1000个
    batch_size=32,         # 训练批次大小：每次迭代处理32个样本
    test_resolutions=[16, 32],  # 测试集数据的分辨率：16x16和32x32
    n_tests=[100, 50],     # 对应不同分辨率测试集的样本数：16x16有100个，32x32有50个
    test_batch_sizes=[32, 32],  # 测试集批次大小：两种分辨率均为32
)


Loading test db for resolution 16 with 100 samples 
Loading test db for resolution 32 with 50 samples 






 ## 创建FNO（Fourier Neural Operator）模型

 傅里叶神经算子是基于傅里叶变换的神经算子，适用于偏微分方程求解等场景



In [11]:
# 初始化FNO模型
model = FNO(
    n_modes=(16, 16),          # 傅里叶模态数：在x和y方向各保留16个模态（模态数越多，拟合能力越强，但计算量越大）
    in_channels=1,             # 输入通道数：1（达西流的渗透率场为单通道）
    out_channels=1,            # 输出通道数：1（达西流的压力场为单通道）
    hidden_channels=32,        # 隐藏层通道数：32（控制模型宽度，影响容量）
    projection_channel_ratio=2, # 投影通道比例：2（用于控制投影层的通道数）
    factorization="tucker",    # 张量分解方式：tucker分解（减少参数量，防止过拟合）
    rank=0.42,                 # 分解秩：0.42（秩越小，参数量越少，分解越激进）
)

# 将模型移到指定设备（CPU/GPU）
model = model.to(device)

# 统计模型参数量
n_params = count_model_params(model)
# 打印参数量信息
print(f"\nOur model has {n_params} parameters.")
# 强制刷新标准输出，确保信息即时打印
sys.stdout.flush()



Our model has 520761 parameters.






 ## 创建优化器和学习率调度器

 优化器用于更新模型参数，调度器用于动态调整学习率



In [12]:
# 初始化AdamW优化器（带权重衰减的Adam，防止过拟合）
optimizer = AdamW(
    model.parameters(),    # 待优化的模型参数
    lr=8e-3,               # 初始学习率：0.008
    weight_decay=1e-4      # 权重衰减系数：0.0001（L2正则化，防止过拟合）
)

# 初始化余弦退火学习率调度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,     # 关联的优化器
    T_max=30       # 学习率周期：30个epoch（学习率在30个epoch内从初始值余弦衰减到0）
)






 ## 创建损失函数

 损失函数用于衡量模型预测值与真实值的差距，指导模型优化



In [13]:
# 初始化L2损失（LpLoss中d=2表示2维数据，p=2即为L2范数）
l2loss = LpLoss(d=2, p=2)
# 初始化H1损失（包含函数值的L2损失 + 梯度的L2损失，更严格的损失度量）
h1loss = H1Loss(d=2)

# 训练损失选择H1Loss（更适合偏微分方程的求解任务）
train_loss = h1loss
# 定义评估损失字典：包含H1和L2两种损失，用于测试阶段评估
eval_losses = {"h1": h1loss, "l2": l2loss}






 ## 打印训练配置信息

 确认模型、优化器、调度器和损失函数的配置



In [14]:
# 打印优化器配置
print("\n### OPTIMIZER ###\n", optimizer)
# 打印学习率调度器配置
print("\n### SCHEDULER ###\n", scheduler)
# 打印损失函数配置
print("\n### LOSSES ###")
print(f"\n * Train: {train_loss}")       # 训练损失
print(f"\n * Test: {eval_losses}")       # 测试损失
# 强制刷新输出
sys.stdout.flush()



### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

 * Train: <neuralop.losses.data_losses.H1Loss object at 0x000001C82B78EAE0>

 * Test: {'h1': <neuralop.losses.data_losses.H1Loss object at 0x000001C82B78EAE0>, 'l2': <neuralop.losses.data_losses.LpLoss object at 0x000001C82B78E660>}






 ## 创建Trainer训练器实例

 Trainer是NeuralOP封装的训练管理器，简化训练流程的实现



In [15]:
trainer = Trainer(
    model=model,                # 待训练的模型
    n_epochs=20,                # 总训练轮数：20个epoch
    device=device,              # 训练设备
    data_processor=data_processor,  # 数据处理器（用于数据的预处理/后处理）
    wandb_log=False,            # 关闭Weights & Biases日志记录（可视化工具）
    eval_interval=3,            # 评估间隔：每3个epoch进行一次测试集评估
    use_distributed=False,      # 关闭分布式训练（单机训练）
    verbose=True,               # 开启详细日志输出（打印训练过程信息）
)






 ## 训练模型

 第一阶段：正常训练并保存断点（checkpoint）

 第二阶段：从保存的断点恢复训练



In [ ]:
# 第一阶段训练：训练并保存断点
trainer.train(
    train_loader=train_loader,  # 训练数据加载器
    test_loaders={},            # 测试数据加载器（本示例暂不使用，传空字典）
    optimizer=optimizer,        # 优化器
    scheduler=scheduler,        # 学习率调度器
    regularizer=False,          # 关闭正则化
    training_loss=train_loss,   # 训练损失函数
    save_every=1,               # 保存间隔：每1个epoch保存一次断点
    save_dir="./checkpoints",   # 断点保存目录：当前路径下的checkpoints文件夹
)

# 第二阶段：从保存的断点（第10个epoch）恢复训练
# 重新创建Trainer实例（也可复用原实例，此处为演示完整流程）
trainer = Trainer(
    model=model,
    n_epochs=20,
    device=device,
    data_processor=data_processor,
    wandb_log=False,
    eval_interval=3,
    use_distributed=False,
    verbose=True,
)

# 从断点目录恢复训练
trainer.train(
    train_loader=train_loader,
    test_loaders={},
    optimizer=optimizer,
    scheduler=scheduler,
    regularizer=False,
    training_loss=train_loss,
    resume_from_dir="./checkpoints",  # 断点恢复目录：读取之前保存的checkpoints文件夹
)

Training on 1000 samples
Testing on [] samples         on resolutions [].
Raw outputs of shape torch.Size([32, 1, 16, 16])
[0] time=5.63, avg_loss=0.6552, train_err=20.4755
Eval: 
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[3] time=5.09, avg_loss=0.2278, train_err=7.1179
Eval: 
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[6] time=5.15, avg_loss=0.1928, train_err=6.0261
Eval: 
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[9] time=5.08, avg_loss=0.1652, train_err=5.1619
Eval: 
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[Rank 0]: saved training state to ./checkpoints
[12] time=6.08, avg_loss=0.1646, train_err=5.1448
Eval: 
[Rank 0]: saved tra

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch._C._nn.gelu was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torch._C._nn.gelu])` or the `torch.serialization.safe_globals([torch._C._nn.gelu])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default